In [1]:
from bspump.jupyter import *
import bspump.kafka
import json

In [2]:
@register_connection
def connection(app):
  return bspump.kafka.KafkaConnection(app, "KafkaConnection")

In [3]:
new_pipeline("Kafka2KafkaPipeline")

In [4]:
@register_source
def kafka_source(app, pipeline):
   return bspump.kafka.KafkaSource(app, pipeline, "KafkaConnection")

In [5]:
sample_events([
    b"""{"foo":"bap"}""",
    b"""{"foo":"baz"}"""
])

In [6]:
@step
def parse_json(event):
    return json.loads(event.decode("utf8"))

{'foo': 'bap'}
{'foo': 'baz'}


In [7]:
@async_step
async def capitalize_foo(inject, event):
    event["foo"] = event["foo"].upper()
    await inject(event)

{'foo': 'BAP'}
{'foo': 'BAZ'}


In [8]:
@async_step
async def reverse_foo(inject, event):
    event["foo"] = "".join(reversed(list(event["foo"])))
    await inject(event)

{'foo': 'PAB'}
{'foo': 'ZAB'}


In [9]:
@step
def encode_json(event):
    return json.dumps(event).encode()

b'{"foo": "PAB"}'
b'{"foo": "ZAB"}'


In [10]:
@register_sink
def init_kafka_sink(app, pipeline):
    return bspump.kafka.KafkaSink(app, pipeline, "KafkaConnection")

In [11]:
end_pipeline()